### Create Model for Participant 1

In [4]:
import os
import numpy as np

In [5]:
# load all image vectors
path_to_features = "/home/aditis/decodingEEG/DecodeEEG/data/feature_vectors"

# Initialize an empty list to store the loaded arrays
data = []

classes = os.listdir(path_to_features+"/training/")

for clas in classes:
    feature_path  = path_to_features + "/training/" + clas
    files = os.listdir(feature_path)
    for file in files:
        if file.endswith('.npy'):  # Ensure it's a .npy file
            file_path = feature_path +"/"+ file
            array = np.load(file_path)  # Load the .npy file
            data.append(array)         # Append to the list

    
# Convert the list of arrays into a matrix
feature_matrix = np.vstack(data)  # Stack arrays vertically

# Verify the shape
print(feature_matrix.shape)  # Output: (16540, 512)



(16540, 512)


In [6]:
# function to convert eeg to spectrogram (17, 401, 75)
from scipy.signal import ShortTimeFFT
from scipy.signal.windows import gaussian
def eeg_to_spectrogram(eeg):
    """
    Converts a single EEG signal (17x100) into its spectrogram.
    """
    g_std = 12  # standard deviation for Gaussian window in samples
    win = gaussian(50, std=g_std, sym=True)  # symmetric Gaussian window
    T_x, N = 1 / 100, 100  # Sampling rate: 100 Hz, signal length: 100 samples
    SFT = ShortTimeFFT(win, hop=2, fs=1/T_x, mfft=800, scale_to='psd')

    # Calculate Spectrogram
    Sx2 = SFT.spectrogram(eeg)  # shape: (17, ?, ?)
    return Sx2

In [1]:
# model architecture
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Build the CNN model
model = models.Sequential()

# Input layer
model.add(layers.InputLayer(input_shape=(17, 401, 75)))

# Convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))  # 32 filters, kernel size 3x3
model.add(layers.MaxPooling2D((2, 2)))  # Max pooling (2x2)

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))  # 64 filters, kernel size 3x3
model.add(layers.MaxPooling2D((2, 2)))  # Max pooling (2x2)

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))  # 128 filters, kernel size 3x3
model.add(layers.MaxPooling2D((2, 2)))  # Max pooling (2x2)

# Flatten the output of the last convolutional layer
model.add(layers.Flatten())

# Dense layers to map to 512-dimensional feature vector
model.add(layers.Dense(512, activation='relu'))  # Dense layer with 512 neurons
model.add(layers.Dropout(0.3))  # Dropout for regularization
model.add(layers.Dense(512, activation='linear'))  # Output layer with 512 neurons (regression task)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Summary of the model architecture
model.summary()



2024-12-12 16:55:44.697903: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-12 16:55:44.891387: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-12 16:55:44.924465: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-12-12 16:55:44.924478: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 17, 401, 32)       21632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 200, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 200, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 100, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 100, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 50, 128)       0

2024-12-12 16:55:46.212800: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-12 16:55:46.212955: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-12-12 16:55:46.212999: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-12-12 16:55:46.213025: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-12-12 16:55:46.213054: W tensorflow/c

In [2]:
# Load eeg data
path_to_eeg = "/home/aditis/decodingEEG/DecodeEEG/data/PreprocessedEEG/sub-01"
data = np.load(path_to_eeg+'/preprocessed_eeg_training.npy', allow_pickle=True).item()
eeg_data = data["preprocessed_eeg_data"]
eeg_data.shape

(16540, 4, 17, 100)

In [17]:
def train_batch_generator(eeg_data, feature_matrix, batch_size=1654):
    """
    Generator for training data batches.
    """
    num_samples = eeg_data.shape[0]

    while True:  # Infinite loop to yield batches
        for i in range(eeg_data.shape[1] - 1):  # Iterate over EEG sets (3 sets)
            eeg = eeg_data[:, i, :, :]
            for k in range(0, num_samples, batch_size):  # Iterate over chunks
                spectro = []
                batch_end = min(k + batch_size, num_samples)  # Ensure no overflow
                for j in range(k, batch_end):
                    Sx2 = eeg_to_spectrogram(eeg[j, :, :])  # Spectrogram calculation
                    spectro.append(Sx2)
                
                spectro = np.stack(spectro)
                features = feature_matrix[k:batch_end, :]
                yield (spectro, features)  # Yield the batch


def validation_batch_generator(eeg_data, feature_matrix, batch_size=1654):
    """
    Generator for validation data batches.
    """
    num_samples = eeg_data.shape[0]
    eeg = eeg_data[:, 3, :, :]  # Use the 4th set for validation

    while True:  # Infinite loop to yield batches
        for k in range(0, num_samples, batch_size):  # Iterate over chunks
            spectro = []
            batch_end = min(k + batch_size, num_samples)  # Ensure no overflow
            for j in range(k, batch_end):
                Sx2 = eeg_to_spectrogram(eeg[j, :, :])  # Spectrogram calculation
                spectro.append(Sx2)
            
            spectro = np.stack(spectro)
            features = feature_matrix[k:batch_end, :]
            yield (spectro, features)  # Yield the batch

 

In [ ]:
# Set parameters
batch_size = 1654
num_samples = eeg_data.shape[0]  # Total samples in dataset
steps_per_epoch = num_samples * 3 // batch_size  # 3 EEG sets for training
validation_steps = num_samples // batch_size    # 1 EEG set for validation
nb_epoch = 10

# Train the model
model.fit(
    train_batch_generator(eeg_data, feature_matrix, batch_size=batch_size),
    epochs=nb_epoch,
    steps_per_epoch=steps_per_epoch,
    verbose=1,
    validation_data=validation_batch_generator(eeg_data, feature_matrix, batch_size=batch_size),
    validation_steps=validation_steps
)

# Save the model
model.save("/home/aditis/decodingEEG/DecodeEEG/data/simple_model_weights/eeg_to_feature_vector_model.h5")


Epoch 1/10
30/30 [==============================] - 511s 17s/step - loss: 1.0174 - mae: 0.6983 - val_loss: 0.7972 - val_mae: 0.6728
Epoch 2/10
30/30 [==============================] - 509s 17s/step - loss: 0.8163 - mae: 0.6560 - val_loss: 0.7894 - val_mae: 0.6484
Epoch 3/10
30/30 [==============================] - ETA: 0s - loss: 0.8044 - mae: 0.6529 